In [7]:
import faiss
import numpy as np
import nbimporter

In [15]:
embeddings = np.load('sample_embeddings.npy').astype('float32')

In [17]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

In [19]:
index.add(embeddings)